# REBEL Reranker Demonstration (Van Gogh Wiki)

This demo showcases how to use [REBELRerank](https://github.com/levinwil/REBEL) to rerank passages. 

REBEL proposes a novel chain-of-thought approach to reranking, which uses query-dependent meta-prompt to find the "most useful" passages (beyond just relevance).

It compares query search results from Van Gogh’s wikipedia with just retrieval (using VectorIndexRetriever from llama-index) and retrieval+reranking with REBEL.


_______________________________
Dependencies:

- **CUDA**
- The built-in retriever, which uses [Pyserini](https://github.com/castorini/pyserini), requires `JDK11`, `PyTorch`, and `Faiss`


### REBEL
Website: [https://github.com/levinwil/REBEL](https://github.com/levinwil/REBEL)\

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.postprocessor import LLMRerank
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [3]:
import os

OPENAI_API_KEY="sk-"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.chunk_size = 512

## Load Data, Build Index

In [5]:
from pathlib import Path
import requests

wiki_titles = [
    "Vincent van Gogh",
]

data_path = Path("data_wiki")
for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

In [6]:
# load documents
documents = SimpleDirectoryReader("./data_wiki/").load_data()

In [7]:
index = VectorStoreIndex.from_documents(
    documents,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


### Retrieval + REBEL Reranking

1. Set up retriever and reranker
2. Retrieve results given search query without reranking
3. Retrieve results given search query with REBEL reranking

In [8]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.core.postprocessor import REBELRerank

import pandas as pd
import torch
from IPython.display import display, HTML


def get_retrieved_nodes(
    query_str,
    llm=None,
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=False,
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)
    retrieved_nodes.reverse()

    if with_reranker:
        # configure reranker
        if llm is None:
            llm = OpenAI(model="gpt-3.5-turbo")
        reranker = REBELRerank(
            llm=llm, top_n=reranker_top_n
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

        # clear cache, rank_zephyr uses 16GB of GPU VRAM
        del reranker
        torch.cuda.empty_cache()

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

### Without `REBEL` reranking

In [9]:
new_nodes = get_retrieved_nodes(
    "Which date did Paul Gauguin arrive in Arles?",
    vector_top_k=50,
    with_reranker=False,
    llm="rank_zephyr",
)

visualize_retrieved_nodes(new_nodes[:3])

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


,Score,Text
0,0.764461,"Between 1885 and his death in 1890, Van Gogh appears to have been building an oeuvre, a collection that reflected his personal vision and could be commercially successful. He was influenced by Blanc's definition of style, that a true painting required optimal use of colour, perspective and brushstrokes. Van Gogh applied the word ""purposeful"" to paintings he thought he had mastered, as opposed to those he thought of as studies. He painted many series of studies; most of which were still lifes, many executed as colour experiments or as gifts to friends. The work in Arles contributed considerably to his oeuvre: those he thought the most important from that time were The Sower, Night Cafe, Memory of the Garden in Etten and Starry Night. With their broad brushstrokes, inventive perspectives, colours, contours and designs, these paintings represent the style he sought.=== Major series ===Van Gogh's stylistic developments are usually linked to the periods he spent living in different places across Europe. He was inclined to immerse himself in local cultures and lighting conditions, although he maintained a highly individual visual outlook throughout. His evolution as an artist was slow and he was aware of his painterly limitations. Van Gogh moved home often, perhaps to expose himself to new visual stimuli, and through exposure develop his technical skill. Art historian Melissa McQuillan believes the moves also reflect later stylistic changes and that Van Gogh used the moves to avoid conflict, and as a coping mechanism for when the idealistic artist was faced with the realities of his then current situation."
1,0.767758,"==== Self-portraits ====Van Gogh created more than 43 self-portraits between 1885 and 1889. They were usually completed in series, such as those painted in Paris in mid-1887, and continued until shortly before his death. Generally the portraits were studies, created during periods when he was reluctant to mix with others or when he lacked models and painted himself.Van Gogh's self-portraits reflect a high degree of self-scrutiny. Often they were intended to mark important periods in his life; for example, the mid-1887 Paris series were painted at the point where he became aware of Claude Monet, Paul Cézanne and Signac. In Self-Portrait with Grey Felt Hat, heavy strains of paint spread outwards across the canvas. It is one of his most renowned self-portraits of that period, ""with its highly organised rhythmic brushstrokes, and the novel halo derived from the Neo-impressionist repertoire was what Van Gogh himself called a 'purposeful' canvas"".They contain a wide array of physiognomical representations. Van Gogh's mental and physical condition is usually apparent; he may appear unkempt, unshaven or with a neglected beard, with deeply sunken eyes, a weak jaw, or having lost teeth. Some show him with full lips, a long face or prominent skull, or sharpened, alert features. His hair is sometimes depicted in a vibrant reddish hue and at other times ash coloured.Van Gogh's self-portraits vary stylistically. In those painted after December 1888, the strong contrast of vivid colours highlight the haggard pallor of his skin. Some depict the artist with a beard, others without. He can be seen with bandages in portraits executed just after he mutilated his ear. In only a few does he depict himself as a painter. Those painted in Saint-Rémy show the head from the right, the side opposite his damaged ear, as he painted himself reflected in his mirror."
2,0.769397,"With their broad brushstrokes, inventive perspectives, colours, contours and designs, these paintings represent the style he sought.=== Major series ===Van Gogh's stylistic developments are usually linked to the periods he spent living in different places across Europe. He was inclined to immerse himself in local cultures and lighting conditions, although he maintained a highly individual visual outlook throughout. His evolution as an artist was s

### With `REBEL` reranking

In [10]:
new_nodes = get_retrieved_nodes(
    "Which date did Paul Gauguin arrive in Arles?",
    vector_top_k=50,
    reranker_top_n=3,
    with_reranker=True,
)

visualize_retrieved_nodes(new_nodes)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTT

,Score,Text
0,1.0,"==== Flowers ====Van Gogh painted several landscapes with flowers, including roses, lilacs, irises, and sunflowers. Some reflect his interests in the language of colour, and also in Japanese ukiyo-e. There are two series of dying sunflowers. The first was painted in Paris in 1887 and shows flowers lying on the ground. The second set was completed a year later in Arles and is of bouquets in a vase positioned in early morning light. Both are built from thickly layered paintwork, which, according to the London National Gallery, evokes the ""texture of the seed-heads"".In these series, Van Gogh was not preoccupied by his usual interest in filling his paintings with subjectivity and emotion; rather, the two series are intended to display his technical skill and working methods to Gauguin, who was about to visit. The 1888 paintings were created during a rare period of optimism for the artist. Vincent wrote to Theo in August 1888:I'm painting with the gusto of a Marseillais eating bouillabaisse, which won't surprise you when it's a question of painting large sunflowers ... If I carry out this plan there'll be a dozen or so panels. The whole thing will therefore be a symphony in blue and yellow. I work on it all these mornings, from sunrise. Because the flowers wilt quickly and it's a matter of doing the whole thing in one go.The sunflowers were painted to decorate the walls in anticipation of Gauguin's visit, and Van Gogh placed individual works around the Yellow House's guest room in Arles. Gauguin was deeply impressed and later acquired two of the Paris versions. After Gauguin's departure, Van Gogh imagined the two major versions of the sunflowers as wings of the Berceuse Triptych, and included them in his Les XX in Brussels exhibit. Today the major pieces of the series are among his best known, celebrated for the sickly connotations of the colour yellow and its tie-in with the Yellow House, the expressionism of the brush strokes, and their contrast against often dark backgrounds."
1,1.0,"With their broad brushstrokes, inventive perspectives, colours, contours and designs, these paintings represent the style he sought.=== Major series ===Van Gogh's stylistic developments are usually linked to the periods he spent living in different places across Europe. He was inclined to immerse himself in local cultures and lighting conditions, although he maintained a highly individual visual outlook throughout. His evolution as an artist was slow and he was aware of his painterly limitations. Van Gogh moved home often, perhaps to expose himself to new visual stimuli, and through exposure develop his technical skill. Art historian Melissa McQuillan believes the moves also reflect later stylistic changes and that Van Gogh used the moves to avoid conflict, and as a coping mechanism for when the idealistic artist was faced with the realities of his then current situation.==== Portraits ====Van Gogh said portraiture was his greatest interest. ""What I'm most passionate about, much much more than all the rest in my profession"", he wrote in 1890, ""is the portrait, the modern portrait."" It is ""the only thing in painting that moves me deeply and that gives me a sense of the infinite."" He wrote to his sister that he wished to paint portraits that would endure, and that he would use colour to capture their emotions and character rather than aiming for photographic realism. Those closest to Van Gogh are mostly absent from his portraits; he rarely painted Theo, van Rappard or Bernard. The portraits of his mother were from photographs.Van Gogh painted Arles' postmaster Joseph Roulin and his family repeatedly. In five versions of La Berceuse (The Lullaby), Van Gogh painted Augustine Roulin quietly holding a rope that rocks the unseen cradle of her infant daughter. Van Gogh had planned for it to be the central image of a triptych, flanked by paintings of sunflowers."
2,1.0,"Vincent Willem van Gogh (Dutch: [ˈvɪnsɛnt ˈʋɪləɱ vɑŋ ˈɣɔx] ; 3